In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
}

In [ ]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "model_pack", ENTITY_NAME: "bento_service"} # bentoservice file from pack step
    ]
    
    # tmp_entities =
    # [
    #     { ENTITY_NAME: "bentoservice_artifacts" } # stored BentoService
    # ]
)

substep.print_interface_info()
substep.exit_in_visualize_mode()

In [ ]:
import numpy as np
import os.path as osp
from pathlib import Path
import json
import atexit
import requests
import base64
import pickle

In [ ]:
# run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

### Loading REST Bentoservice 

In [ ]:
from sinara.bentoml import load_bentoservice

# load bentoservise
inputs_model_pack = substep.inputs(step_name = "model_pack")
bento_service = load_bentoservice(inputs_model_pack.bento_service)

In [ ]:
bento_service.service_version()

### Start Bentoservice

In [ ]:
from sinara.bentoml import start_dev_bentoservice, stop_dev_bentoservice

In [ ]:
%%capture cap --no-stderr
# Stop a dev model server if running
stop_dev_bentoservice(bento_service)

# Start a dev model server to test out the API endpoint locally
start_dev_bentoservice(bento_service, use_popen=False, debug=False)
_=atexit.register(stop_dev_bentoservice)

In [ ]:
# example REST API get service_version
service_version = requests.post("http://127.0.0.1:5000/service_version", json={}).json()
print(f"service_version: {service_version}")

### Test 1. Predict the received test_data and compare it with test_result

#### Get test_data   
(test image, which should be stored in the bento service, and can be obtained using the test_data method)

In [ ]:
test_data_api_endpoint = f'http://127.0.0.1:5000/test_data'
test_data_response = requests.request("POST", test_data_api_endpoint, json={})

content = test_data_response.json()
encoded_test_data = content['b64']
test_data = base64.b64decode(encoded_test_data)

#### Get test_result   
(the result obtained from the test image and saved in the bento service - stored as pickle, method - test_result)

In [ ]:
test_result_api_endpoint = f'http://127.0.0.1:5000/test_result'
test_result_response = requests.request("POST", test_result_api_endpoint, json={})

content = test_result_response.json()
encoded_test_result = content['b64']
decoded_test_result = base64.b64decode(encoded_test_result)

test_result = pickle.loads(decoded_test_result)

#### Predict test_data
(sending test image test_data to predict method)

In [ ]:
predict_api_endpoint = f'http://127.0.0.1:5000/predict'

headers = {
  'Content-Type': 'application/octet-stream'
}

predict_response = requests.post(predict_api_endpoint, headers=headers, data=test_data)
predict_result = predict_response.json()

#### Compare results of predict by the test_data and test_result 

In [ ]:
def bb_intersection_over_union(coco_boxA, coco_boxB):
    # determine the (x_top_left, y_top_left, h, w)-coordinates of the intersection rectangle
    boxA = [coco_boxA[0], coco_boxA[1], coco_boxA[0]+coco_boxA[2], coco_boxA[1]+coco_boxA[3]]
    boxB = [coco_boxB[0], coco_boxB[1], coco_boxB[0]+coco_boxB[2], coco_boxB[1]+coco_boxB[3]]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [ ]:
_success = True

try:
    for predict_obj in predict_result["annotations"]:
        predict_obj_category_id = predict_obj["category_id"]
        predict_obj_bbox = predict_obj["bbox"]
        ious = []
        for test_obj in test_result["annotations"]:
            test_obj_category_id = test_obj["category_id"]
            
            if predict_obj_category_id != test_obj_category_id:  # comparison by object class
                continue
            test_obj_bbox = test_obj["bbox"]
            ious.append(bb_intersection_over_union(np.array(predict_obj_bbox), test_obj_bbox))    
        
        id_bbox_iou = np.array(ious).argmax()  
        if ious[id_bbox_iou] < 0.98:  # no matches by iou     
            _success = False
except Exception as e:
    print(f"error: {e.__str__()}")
    _success = False

In [ ]:
assert _success

In [ ]:
# Stop spark
SinaraSpark.stop_session()